# Projeto

## Data Preparation

### 3 Alternativas (mudar para 3 se houver tempo):
1. 

2. 

3. 

In [3]:
import pandas as pd

filename = 'data_preparation_csvs/mv_imputation_csvs/data_mv_alt1.csv'
data_mv_imputation: pd.DataFrame = pd.read_csv(filename, sep=',', decimal='.', na_values='')
print(f"Dataset nr records={data_mv_imputation.shape[0]}", f"nr variables={data_mv_imputation.shape[1]}")
data_mv_imputation.columns

Dataset nr records=99934 nr variables=31


Index(['Unnamed: 0', 'Month', 'Occupation', 'Payment_of_Min_Amount',
       'Payment_Behaviour', 'Payday Loan', 'Personal Loan',
       'Debt Consolidation Loan', 'Auto Loan', 'Not Specified Loan',
       'Student Loan', 'Credit-Builder Loan', 'Mortgage Loan',
       'Home Equity Loan', 'Credit_Score', 'Age', 'Annual_Income',
       'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'NumofLoan',
       'Delay_from_due_date', 'NumofDelayedPayment', 'ChangedCreditLimit',
       'NumCreditInquiries', 'OutstandingDebt', 'CreditUtilizationRatio',
       'Credit_History_Age', 'TotalEMIpermonth', 'Amountinvestedmonthly',
       'MonthlyBalance'],
      dtype='object')